In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install -U ultralytics "ray[tune]==2.10.0" wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Enable W&B logging for Ultralytics

!yolo settings wandb=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [ ]:
import wandb

# Log in to Weights & Biases (use your API key)
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: my_object_detection_project (my_object_detection_project-democritus-university-of-thrace) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# EXP 1

from ultralytics import YOLO
from ray import tune
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize a W&B run with resume option
wandb.init(project="Hyperparameter_Tuning", job_type="tuning", name="small_tune")

# Load a YOLO model
model = YOLO("yolov8s.pt")

# Define your custom search space
search_space = {
        "momentum": tune.uniform(0.6, 0.98),
        "weight_decay": tune.uniform(0.0, 0.001),
        "warmup_epochs": tune.uniform(0.0, 5.0),
        "warmup_momentum": tune.uniform(0.0, 0.95),
        "warmup_bias_lr": tune.uniform(0.0, 0.2),
        "box": tune.uniform(0.02, 0.2),
        "cls": tune.uniform(0.2, 4.0),
        "dfl": tune.uniform(0.4, 6.0),
}

# Start tuning the model using Ray Tune
result_grid = model.tune(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    space=search_space,
    epochs=10,
    use_ray=True,
    gpu_per_trial=1,
    grace_period=5,
    iterations=10,
)

wandb.finish()

In [ ]:
# EXP 2

from ultralytics import YOLO
from ray import tune
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize a W&B run with resume option
wandb.init(project="Hyperparameter_Tuning", group="medium_tune_exp2", job_type="tuning", name="medium_tune_exp2")

# Load a YOLO model
model = YOLO("yolov8m.pt")

# Define your custom search space
search_space = {
        "momentum": tune.uniform(0.6, 0.98),
        "weight_decay": tune.uniform(0.0, 0.001),
        "warmup_epochs": tune.uniform(0.0, 5.0),
        "warmup_momentum": tune.uniform(0.0, 0.95),
        "warmup_bias_lr": tune.uniform(0.0, 0.2),
        "box": tune.uniform(0.02, 0.2),
        "cls": tune.uniform(0.2, 4.0),
        "dfl": tune.uniform(0.4, 6.0),
}

# Start tuning the model using Ray Tune
result_grid = model.tune(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    space=search_space,
    epochs=10,
    use_ray=True,
    gpu_per_trial=1,
    grace_period=5,
    iterations=10,
)

wandb.finish()

In [ ]:
# EXP 3

from ultralytics import YOLO
from ray import tune
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize a W&B run with resume option
wandb.init(project="Hyperparameter_Tuning", group="medium_tune_exp3", job_type="tuning", name="medium_tune_exp3")

# Load a YOLO model
model = YOLO("yolov8m.pt")

# Define your custom search space
search_space = {
    "momentum": tune.uniform(0.7, 0.9),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "warmup_epochs": tune.quniform(0.5, 5.0, 0.5),
    "warmup_momentum": tune.uniform(0.4, 0.8),
    "warmup_bias_lr": tune.quniform(0.05, 0.2, 0.01),
    "box": tune.uniform(0.03, 0.1),
    "cls": tune.uniform(0.2, 1.0),
    "dfl": tune.uniform(0.5, 2.0),
}

# Start tuning the model using Ray Tune
result_grid = model.tune(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    space=search_space,
    use_ray=True,
    iterations=10,
    epochs=10,
    gpu_per_trial=1,
    device='cuda',
    grace_period=5,
    patience=5,
)

wandb.finish()

In [ ]:
# EXP 4

from ultralytics import YOLO
from ray import tune
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize a W&B run with resume option
wandb.init(project="Hyperparameter_Tuning", group="medium_tune_exp4", job_type="tuning", name="medium_tune_exp4")

# Load a YOLO model
model = YOLO("yolov8m.pt")

# Define your custom search space
search_space = {
    "momentum": tune.uniform(0.75, 0.9),
    "weight_decay": tune.loguniform(1e-6, 5e-3),
    "warmup_epochs": tune.quniform(1.0, 5.0, 0.5),
    "warmup_momentum": tune.uniform(0.5, 0.8),
    "warmup_bias_lr": tune.quniform(0.08, 0.18, 0.01),
    "box": tune.uniform(0.04, 0.09),
    "cls": tune.uniform(0.3, 0.8),
    "dfl": tune.uniform(0.7, 1.5),
}

# Start tuning the model using Ray Tune
result_grid = model.tune(
    data='/content/drive/My Drive/Colab Notebooks/Final_Dataset/Dataset.yaml',
    space=search_space,
    use_ray=True,
    iterations=10,
    epochs=10,
    gpu_per_trial=1,
    device='cuda',
    grace_period=5,
    patience=5,
)

wandb.finish()

Output hidden; open in https://colab.research.google.com to view.